<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/fasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keystone-engine
!pip install capstone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.7 MB/s eta 0:00:00


In [2]:
from keystone import Ks, KS_ARCH_X86, KS_MODE_32

# Инициализация ассемблера
ks = Ks(KS_ARCH_X86, KS_MODE_32)

CODE = """
    mov eax, 3;
"""
encoding, count = ks.asm(CODE)
machine_code = bytes(encoding)

print(f"Машинный код: {machine_code}")

Машинный код: b'\xb8\x03\x00\x00\x00'


In [3]:
from capstone import Cs, CS_ARCH_X86, CS_MODE_32

# Инициализация дизассемблера
md = Cs(CS_ARCH_X86, CS_MODE_32)

# Дизассемблирование машинного кода
for instruction in md.disasm(machine_code, 0x1000):
    print(f"0x{instruction.address:x}: {instruction.mnemonic} {instruction.op_str}")

0x1000: mov eax, 3


In [4]:
!wget https://flatassembler.net/fasm-1.73.32.tgz
!tar -xvzf fasm-1.73.32.tgz

--2025-03-14 21:00:41--  https://flatassembler.net/fasm-1.73.32.tgz
Resolving flatassembler.net (flatassembler.net)... 208.99.203.253
Connecting to flatassembler.net (flatassembler.net)|208.99.203.253|:443... connected.
HTTP request sent, awaiting response... 200 She'll be apples
Length: 350082 (342K) [application/octet-stream]
Saving to: ‘fasm-1.73.32.tgz’

fasm-1.73.32.tgz    100%[===================>] 341.88K  --.-KB/s    in 0.1s    

2025-03-14 21:00:41 (2.36 MB/s) - ‘fasm-1.73.32.tgz’ saved [350082/350082]

fasm/
fasm/examples/
fasm/examples/elfexe/
fasm/examples/elfexe/dynamic/
fasm/examples/elfexe/dynamic/elf.inc
fasm/examples/elfexe/dynamic/hello.asm
fasm/examples/elfexe/dynamic/hello64.asm
fasm/examples/elfexe/dynamic/import32.inc
fasm/examples/elfexe/dynamic/import64.inc
fasm/examples/elfexe/dynamic/proc32.inc
fasm/examples/elfexe/hello.asm
fasm/examples/elfexe/hello64.asm
fasm/examples/elfobj/
fasm/examples/elfobj/msgdemo.asm
fasm/examples/elfobj/writemsg.asm
fasm/examples/l

In [5]:
!cat fasm/examples/elfexe/hello.asm


; fasm demonstration of writing simple ELF executable

format ELF executable 3
entry start

segment readable executable

start:

	mov	eax,4
	mov	ebx,1
	mov	ecx,msg
	mov	edx,msg_size
	int	0x80

	mov	eax,1
	xor	ebx,ebx
	int	0x80

segment readable writeable

msg db 'Hello world!',0xA
msg_size = $-msg


In [6]:
!fasm/fasm fasm/examples/elfexe/hello.asm hello

flat assembler  version 1.73.32  (16384 kilobytes memory)
3 passes, 160 bytes.


In [7]:
!./hello

Hello world!


In [33]:
%%file power.asm
format ELF64 executable  ; Указываем формат выходного файла (ELF64 для 64-битной архитектуры)
entry start              ; Точка входа в программу

segment readable executable  ; Секция кода

start:
    ; Выводим сообщение "Введите число: "
    mov rax, 1          ; syscall: write
    mov rdi, 1          ; file descriptor: stdout
    mov rsi, msg_num    ; адрес сообщения
    mov rdx, msg_num_len ; длина сообщения
    syscall             ; вызов системного вызова

    ; Считываем число из консоли
    mov rax, 0          ; syscall: read
    mov rdi, 0          ; file descriptor: stdin
    mov rsi, num_buffer ; адрес буфера для ввода
    mov rdx, 10         ; максимальная длина ввода (10 символов)
    syscall             ; вызов системного вызова

    ; Преобразуем введенную строку в число
    mov rcx, num_buffer ; адрес буфера
    call atoi           ; вызываем функцию atoi
    mov [num], rax      ; сохраняем число

    ; Выводим сообщение "Введите степень числа: "
    mov rax, 1          ; syscall: write
    mov rdi, 1          ; file descriptor: stdout
    mov rsi, msg_pow    ; адрес сообщения
    mov rdx, msg_pow_len ; длина сообщения
    syscall             ; вызов системного вызова

    ; Считываем степень из консоли
    mov rax, 0          ; syscall: read
    mov rdi, 0          ; file descriptor: stdin
    mov rsi, pow_buffer ; адрес буфера для ввода
    mov rdx, 10         ; максимальная длина ввода (10 символов)
    syscall             ; вызов системного вызова

    ; Преобразуем введенную строку в число
    mov rcx, pow_buffer ; адрес буфера
    call atoi           ; вызываем функцию atoi
    mov [pow], rax      ; сохраняем степень

    ; Возводим число в степень
    mov rax, [num]      ; загружаем число
    mov rcx, [pow]      ; загружаем степень
    call power          ; вызываем функцию power
    mov [result], rax   ; сохраняем результат

    ; Преобразуем результат в строку
    mov rax, [result]   ; загружаем результат
    mov rcx, result_str ; адрес буфера для строки
    call itoa           ; вызываем функцию itoa

    ; Выводим сообщение "Результат возведения в степень: "
    mov rax, 1          ; syscall: write
    mov rdi, 1          ; file descriptor: stdout
    mov rsi, msg_result ; адрес сообщения
    mov rdx, msg_result_len ; длина сообщения
    syscall             ; вызов системного вызова

    ; Выводим результат
    mov rax, 1          ; syscall: write
    mov rdi, 1          ; file descriptor: stdout
    mov rsi, result_str ; адрес строки результата
    mov rdx, 20         ; длина вывода (20 символов)
    syscall             ; вызов системного вызова

    ; Завершаем программу
    mov rax, 60         ; syscall: exit
    xor rdi, rdi        ; exit code 0
    syscall             ; вызов системного вызова

; Функция atoi: преобразует строку в число
atoi:
    xor rax, rax        ; обнуляем rax
    xor rbx, rbx        ; обнуляем rbx
atoi_loop:
    mov bl, [rcx]       ; загружаем символ из строки
    cmp bl, 0xA         ; проверяем на символ новой строки
    je atoi_done        ; если символ новой строки, завершаем
    sub bl, '0'         ; преобразуем символ в число
    imul rax, 10        ; умножаем текущее значение на 10
    add rax, rbx        ; добавляем новое число
    inc rcx             ; переходим к следующему символу
    jmp atoi_loop       ; повторяем
atoi_done:
    ret

; Функция itoa: преобразует число в строку
itoa:
    mov rbx, 10         ; делитель (основание системы счисления)
    xor rdx, rdx        ; обнуляем rdx
    mov rdi, rcx        ; сохраняем адрес буфера
    add rdi, 19         ; перемещаемся в конец буфера
    mov byte [rdi], 0   ; добавляем нулевой символ в конец строки
    dec rdi             ; перемещаемся на одну позицию назад
itoa_loop:
    xor rdx, rdx        ; обнуляем rdx
    div rbx             ; делим rax на 10
    add dl, '0'         ; преобразуем остаток в символ
    mov [rdi], dl       ; сохраняем символ в буфер
    dec rdi             ; перемещаемся на одну позицию назад
    test rax, rax       ; проверяем, осталось ли число
    jnz itoa_loop       ; если число не нулевое, повторяем
    ret

; Функция power: возводит число в степень
power:
    mov rax, 1          ; инициализируем результат (1)
power_loop:
    test rcx, rcx       ; проверяем, равна ли степень нулю
    jz power_done       ; если степень равна нулю, завершаем
    imul rax, [num]     ; умножаем результат на число
    dec rcx             ; уменьшаем степень на 1
    jmp power_loop      ; повторяем
power_done:
    ret

segment readable writeable  ; Секция данных

msg_num db 'Введите число: ', 0
msg_num_len = $ - msg_num

msg_pow db 'Введите степень числа: ', 0
msg_pow_len = $ - msg_pow

msg_result db 'Результат возведения в степень: ', 0
msg_result_len = $ - msg_result

num_buffer db 0, 0, 0, 0, 0, 0, 0, 0, 0, 0  ; Буфер для ввода числа
pow_buffer db 0, 0, 0, 0, 0, 0, 0, 0, 0, 0  ; Буфер для ввода степени
result_str db 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0  ; Буфер для вывода результата

num dq 0  ; Переменная для числа (64-битное значение)
pow dq 0  ; Переменная для степени (64-битное значение)
result dq 0  ; Переменная для результата (64-битное значение)

Overwriting power.asm


In [34]:
!fasm/fasm power.asm power

flat assembler  version 1.73.32  (16384 kilobytes memory)
3 passes, 744 bytes.


In [35]:
!./power

Введите число:  2
Введите степень числа:  50
Результат возведения в степень:     1125899906842624 